# 1. Preprocess Data

In [ ]:
import pandas as pd
import numpy as np
import nltk
from nltk.stem import WordNetLemmatizer
import re

In [ ]:
!gdown '1cXAo2k0dJnFimY5EcmWwd7vnCwrtV_tt'

Downloading...
From: https://drive.google.com/uc?id=1cXAo2k0dJnFimY5EcmWwd7vnCwrtV_tt
To: /content/IMDB Dataset.csv
100% 66.2M/66.2M [00:01<00:00, 37.7MB/s]


Đọc dữ liệu vào

In [ ]:
df = pd.read_csv('IMDB Dataset.csv')

Tìm hiểu về dữ liệu

In [ ]:
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review     50000 non-null  object
 1   sentiment  50000 non-null  object
dtypes: object(2)
memory usage: 781.4+ KB


In [ ]:
df['sentiment'].value_counts()

sentiment
positive    25000
negative    25000
Name: count, dtype: int64

Nhận xét: dữ liệu cân bằng với cả "negative" và "positive"

In [ ]:
df['review'][1]

'A wonderful little production. <br /><br />The filming technique is very unassuming- very old-time-BBC fashion and gives a comforting, and sometimes discomforting, sense of realism to the entire piece. <br /><br />The actors are extremely well chosen- Michael Sheen not only "has got all the polari" but he has all the voices down pat too! You can truly see the seamless editing guided by the references to Williams\' diary entries, not only is it well worth the watching but it is a terrificly written and performed piece. A masterful production about one of the great master\'s of comedy and his life. <br /><br />The realism really comes home with the little things: the fantasy of the guard which, rather than use the traditional \'dream\' techniques remains solid then disappears. It plays on our knowledge and our senses, particularly with the scenes concerning Orton and Halliwell and the sets (particularly of their flat with Halliwell\'s murals decorating every surface) are terribly well d

Thực hiện xóa những html tags và những nội dung trong dấu ngoặc vuông

In [ ]:
from bs4 import BeautifulSoup
def xoa_html(text):
    soup = BeautifulSoup(text, "html.parser")
    return soup.get_text()

def xoa_dau_ngoac(text):
    return re.sub('\[[^]]*\]', '', text)

def denoise_text(text):
    text = xoa_html(text)
    text = xoa_dau_ngoac(text)
    return text

df['review']=df['review'].apply(denoise_text)

<ipython-input-8-69edd58c9e71>:3: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(text, "html.parser")


Xóa tiếp những kí tự không phải là chữ cái và chữ số

In [ ]:
def xoa_ki_tu(text, remove_digits=True):
    pattern=r'[^a-zA-z0-9\s]'
    text=re.sub(pattern,'',text)
    return text

df['review'] = df['review'].apply(xoa_ki_tu)
df['review'][1]

'A wonderful little production The filming technique is very unassuming very oldtimeBBC fashion and gives a comforting and sometimes discomforting sense of realism to the entire piece The actors are extremely well chosen Michael Sheen not only has got all the polari but he has all the voices down pat too You can truly see the seamless editing guided by the references to Williams diary entries not only is it well worth the watching but it is a terrificly written and performed piece A masterful production about one of the great masters of comedy and his life The realism really comes home with the little things the fantasy of the guard which rather than use the traditional dream techniques remains solid then disappears It plays on our knowledge and our senses particularly with the scenes concerning Orton and Halliwell and the sets particularly of their flat with Halliwells murals decorating every surface are terribly well done'

Loại bỏ các stopwords để giảm bớt kích thước dữ liệu

In [ ]:
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
df['review'] = df['review'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))
df['review'][1]

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


'A wonderful little production The filming technique unassuming oldtimeBBC fashion gives comforting sometimes discomforting sense realism entire piece The actors extremely well chosen Michael Sheen got polari voices pat You truly see seamless editing guided references Williams diary entries well worth watching terrificly written performed piece A masterful production one great masters comedy life The realism really comes home little things fantasy guard rather use traditional dream techniques remains solid disappears It plays knowledge senses particularly scenes concerning Orton Halliwell sets particularly flat Halliwells murals decorating every surface terribly well done'

Lemmatize văn bản

In [ ]:
nltk.download('wordnet')
w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
lemmatizer = nltk.stem.WordNetLemmatizer()
def lemmatize_text(text):
    st = ""
    for w in w_tokenizer.tokenize(text):
        st = st + lemmatizer.lemmatize(w) + " "
    return st
df['review'] = df.review.apply(lemmatize_text)
df['review'][1]

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


'A wonderful little production The filming technique unassuming oldtimeBBC fashion give comforting sometimes discomforting sense realism entire piece The actor extremely well chosen Michael Sheen got polari voice pat You truly see seamless editing guided reference Williams diary entry well worth watching terrificly written performed piece A masterful production one great master comedy life The realism really come home little thing fantasy guard rather use traditional dream technique remains solid disappears It play knowledge sens particularly scene concerning Orton Halliwell set particularly flat Halliwells mural decorating every surface terribly well done '

Chia bộ dữ liệu thành các tập train và test sau đó kiểm tra độ cân bằng của bộ dữ liệu

In [ ]:
X_train = df['review'][:40000]
y_train = df['sentiment'][:40000]
X_test = df['review'][40000:]
y_test = df['sentiment'][40000:]
print(y_train.value_counts(), y_test.value_counts())

sentiment
negative    20007
positive    19993
Name: count, dtype: int64 sentiment
positive    5007
negative    4993
Name: count, dtype: int64


Bộ train và test khá cân bằng

# 2. Feature Extraction

## a. Bow


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

# Tạo đối tượng CountVectorizer
vectorizer = CountVectorizer(max_features=5000)  # Giới hạn số lượng từ đặc trưng

# Áp dụng BoW lên dữ liệu huấn luyện và kiểm tra
X_train_bow = vectorizer.fit_transform(X_train)
X_test_bow = vectorizer.transform(X_test)

print("BoW X_train shape:", X_train_bow.shape)
print("BoW X_test shape:", X_test_bow.shape)

BoW X_train shape: (40000, 5000)
BoW X_test shape: (10000, 5000)


## b. TF-IDF

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Tạo đối tượng TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer(max_features=5000)

# Áp dụng TF-IDF lên dữ liệu huấn luyện và kiểm tra
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

print("TF-IDF X_train shape:", X_train_tfidf.shape)
print("TF-IDF X_test shape:", X_test_tfidf.shape)

TF-IDF X_train shape: (40000, 5000)
TF-IDF X_test shape: (10000, 5000)


## c. Word Embeddings


In [ ]:
from gensim.models import Word2Vec

# Chuẩn bị dữ liệu cho Word2Vec
sentences = [text.split() for text in df['review']]

# Huấn luyện mô hình Word2Vec
word2vec_model = Word2Vec(sentences, vector_size=100, window=5, min_count=1, workers=4)

# Lấy vector embedding cho mỗi từ
def get_embedding_vector(word):
    if word in word2vec_model.wv:
        return word2vec_model.wv[word]
    else:
        return np.zeros(word2vec_model.vector_size)

# Hàm để tạo vector cho mỗi review
def get_sentence_embedding(text):
    words = text.split()
    vectors = [get_embedding_vector(word) for word in words]
    if vectors:
        return np.mean(vectors, axis=0)
    else:
        return np.zeros(word2vec_model.vector_size)

# Áp dụng Word2Vec lên dữ liệu huấn luyện và kiểm tra
X_train_word2vec = np.array([get_sentence_embedding(text) for text in X_train])
X_test_word2vec = np.array([get_sentence_embedding(text) for text in X_test])

print("Embeddings X_train shape:", X_train_word2vec.shape)
print("Embeddings X_test shape:", X_test_word2vec.shape)

Embeddings X_train shape: (40000, 100)
Embeddings X_test shape: (10000, 100)


Bạn có thể sử dụng hoặc thử cả 3 cách biểu diễn trên để train model


# 3. Training and Evaluating model

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score

## a. Bow

In [ ]:
# Logistic Regression
logreg_bow = LogisticRegression(max_iter=1000)
logreg_bow.fit(X_train_bow, y_train)

y_pred_logreg_bow = logreg_bow.predict(X_test_bow)
print("Logistic Regression (Bow) Accuracy:", accuracy_score(y_test, y_pred_logreg_bow))
print("Logistic Regression (Bow) Classification Report:")
print(classification_report(y_test, y_pred_logreg_bow))

Logistic Regression (Bow) Accuracy: 0.8726
Logistic Regression (Bow) Classification Report:
              precision    recall  f1-score   support

    negative       0.87      0.87      0.87      4993
    positive       0.87      0.87      0.87      5007

    accuracy                           0.87     10000
   macro avg       0.87      0.87      0.87     10000
weighted avg       0.87      0.87      0.87     10000



--Số liệu dương và âm của precision, recall, f1-score trong mô hình Logistic Regression đều có số liệu tương đương nhau.

Đánh giá: Không quá tốn thời gian

In [ ]:
# SVM
svm_bow = SVC()
svm_bow.fit(X_train_bow, y_train)

y_pred_svm_bow = svm_bow.predict(X_test_bow)
print("SVM (Bow) Accuracy:", accuracy_score(y_test, y_pred_svm_bow))
print("SVM (Bow) Classification Report:")
print(classification_report(y_test, y_pred_svm_bow))

SVM (Bow) Accuracy: 0.8746
SVM (Bow) Classification Report:
              precision    recall  f1-score   support

    negative       0.89      0.86      0.87      4993
    positive       0.86      0.89      0.88      5007

    accuracy                           0.87     10000
   macro avg       0.88      0.87      0.87     10000
weighted avg       0.88      0.87      0.87     10000



Tốn cỡ 40 phút

-Số liệu dương và âm của precision, recall, f1-score trong mô hình SVM đều có số liệu tương đương nhau.

## b. TF-IDF

In [ ]:
# Logistic Regression
logreg_tfidf = LogisticRegression(max_iter=1000)
logreg_tfidf.fit(X_train_tfidf, y_train)

y_pred_logreg_tfidf = logreg_tfidf.predict(X_test_tfidf)
print("Logistic Regression (TF-IDF) Accuracy:", accuracy_score(y_test, y_pred_logreg_tfidf))
print("Logistic Regression (TF-IDF) Classification Report:")
print(classification_report(y_test, y_pred_logreg_tfidf))

Logistic Regression (TF-IDF) Accuracy: 0.8861
Logistic Regression (TF-IDF) Classification Report:
              precision    recall  f1-score   support

    negative       0.89      0.88      0.89      4993
    positive       0.88      0.89      0.89      5007

    accuracy                           0.89     10000
   macro avg       0.89      0.89      0.89     10000
weighted avg       0.89      0.89      0.89     10000



-Số liệu dương và âm của precision, recall, f1-score trong mô hình Logistic Regression đều có số liệu tương đương nhau.

In [ ]:
# SVM
svm_tfidf = SVC()
svm_tfidf.fit(X_train_tfidf, y_train)
y_pred_svm_tfidf = svm_tfidf.predict(X_test_tfidf)

print("SVM (TF-IDF) Accuracy:", accuracy_score(y_test, y_pred_svm_tfidf))
print("SVM (TF-IDF) Classification Report:")
print(classification_report(y_test, y_pred_svm_tfidf))

SVM (TF-IDF) Accuracy: 0.8884
SVM (TF-IDF) Classification Report:
              precision    recall  f1-score   support

    negative       0.89      0.88      0.89      4993
    positive       0.88      0.89      0.89      5007

    accuracy                           0.89     10000
   macro avg       0.89      0.89      0.89     10000
weighted avg       0.89      0.89      0.89     10000



-Số liệu dương và âm của precision, recall, f1-score trong mô hình SVM đều có số liệu tương đương nhau.

## c. Word2Vec Embeddings

In [ ]:
# Logistic Regression
logreg_word2vec = LogisticRegression(max_iter=1000)
logreg_word2vec.fit(X_train_word2vec, y_train)

y_pred_logreg_word2vec = logreg_word2vec.predict(X_test_word2vec)
print("Logistic Regression (Word2Vec) Accuracy:", accuracy_score(y_test, y_pred_logreg_word2vec))
print("Logistic Regression (Word2Vec) Classification Report:")
print(classification_report(y_test, y_pred_logreg_word2vec))

Logistic Regression (Word2Vec) Accuracy: 0.8535
Logistic Regression (Word2Vec) Classification Report:
              precision    recall  f1-score   support

    negative       0.86      0.85      0.85      4993
    positive       0.85      0.86      0.85      5007

    accuracy                           0.85     10000
   macro avg       0.85      0.85      0.85     10000
weighted avg       0.85      0.85      0.85     10000



-Số liệu dương và âm của precision, recall, f1-score trong mô hình Logistic Regression đều có số liệu tương đương nhau.

In [ ]:
# SVM
svm_word2vec = SVC()
svm_word2vec.fit(X_train_word2vec, y_train)

y_pred_svm_word2vec = svm_word2vec.predict(X_test_word2vec)
print("SVM (Word2Vec) Accuracy:", accuracy_score(y_test, y_pred_svm_word2vec))
print("SVM (Word2Vec) Classification Report:")
print(classification_report(y_test, y_pred_svm_word2vec))

SVM (Word2Vec) Accuracy: 0.8575
SVM (Word2Vec) Classification Report:
              precision    recall  f1-score   support

    negative       0.86      0.85      0.86      4993
    positive       0.86      0.86      0.86      5007

    accuracy                           0.86     10000
   macro avg       0.86      0.86      0.86     10000
weighted avg       0.86      0.86      0.86     10000



-Số liệu dương và âm của precision, recall, f1-score trong mô hình SVM đều có số liệu tương đương nhau.